# Refactor Network Charts 

Refactor network charts code to place geometry placement code in its own "mixin"...

In [1]:
import os
from collections import defaultdict
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from dmt.tk.plotting.shapes import Circle, Arc, PolarPoint
from dmt.tk.plotting.charts.network import CircularNetworkChart

In [2]:
N = 6
mtypes = ["L23_MC", "L5_TPC:A", "L6_UPC", "L23_BC", "L1_SBC", "L4_TPC"][:N]
sclasses = pd.Series(
    ["INH", "EXC", "EXC", "INH", "INH", "EXC"][:N],
    index=mtypes, name="sclass")
pathways = [
    (pre_mtype, post_mtype)
    for pre_mtype in mtypes
    for post_mtype in mtypes]
N = len(pathways)
pathway_strength = pd.DataFrame(
    {("pathway_strength", "mean"): np.random.uniform(size=N),
     ("pathway_strength", "std"):  np.random.uniform(size=N)},
    index=pd.MultiIndex.from_tuples(
        pathways,
        names=["pre_mtype", "post_mtype"]))
def get_node_weights(dataframe):
    weights = dataframe.groupby("pre_mtype").agg("sum")[("pathway_strength", "mean")]
    weights.index.name = "label"
    weights.name = "weight"
    return weights/np.sum(weights)

link_weights = pd.Series(
    pathway_strength[("pathway_strength", "mean")].values,
    index = pathway_strength.index.rename(["begin_node", "end_node"]),
    name="weight")

In [3]:
layer_cols = {'L1': np.array([0.3, 0.3, 0]),
              'L2': np.array([0.0, 0.3, 0.3]),
              'L3': np.array([0.15, 0.3, 0.15]),
              'L4': np.array([0.3, 0.0, 0.3]),
              'L5': np.array([0.0, 0.6, 0.0]),
              'L6': np.array([0.1, 0.1, 0.4])}
layer_cols["L23"] = (layer_cols["L2"] + layer_cols["L3"]) / 2.

default = np.array([0.2, 0.2, 0.2])
add_ei = {'EXC': np.array([0.5, 0.0, 0.0]),
          'INH': np.array([0.0, 0.0, 0.5])}


def _get_color(mtype, sclass):
    layer = mtype.split('_')[0]
    return np.minimum(
        layer_cols[layer]
            + add_ei[sclass] 
            + np.random.rand(3) * 0.25, 1.0)
def make_col_dict(mtypes):
    return dict([(_m[0], numpy.minimum(layer_cols.setdefault(_m[0][1], default)
                                       + add_ei[_m[1]] +
                                       numpy.random.rand(3) * 0.25, 1.0))
                 for _m in mtypes])

In [4]:
mtypes = link_weights.index.get_level_values("begin_node").unique()
color_map ={
    mtype: _get_color(mtype, sclass)
    for mtype, sclass in sclasses.items()}
for pre in mtypes:
    for post in mtypes:
        link_color = list(color_map[pre])
        link_color.append(0.25)
        color_map[(pre, post)] = np.array([0.8, 0.8, 0.8, 1.0]) * link_color
chart =\
    CircularNetworkChart(
        link_data=link_weights,
        color_map=color_map)

In [5]:
chart.link_data.groupby(
    "begin_node"
).agg(
    "sum"
).rename(
    "outgoing"
)

begin_node
L1_SBC      2.313706
L23_BC      2.295342
L23_MC      2.053800
L4_TPC      3.150950
L5_TPC:A    2.474294
L6_UPC      3.628422
Name: outgoing, dtype: float64

In [6]:
chart.node_flow

,outgoing,incoming,total
L1_SBC,2.313706,2.383248,4.696953
L23_BC,2.295342,2.419059,4.714401
L23_MC,2.053800,3.143935,5.197736
L4_TPC,3.150950,2.185307,5.336257
L5_TPC:A,2.474294,3.193901,5.668195
L6_UPC,3.628422,2.591064,6.219487


In [7]:
chart.node_size

,total,source,target
L1_SBC,4.696953,2.313706,2.383248
L23_BC,4.714401,2.295342,2.419059
L23_MC,5.197736,2.053800,3.143935
L4_TPC,5.336257,3.150950,2.185307
L5_TPC:A,5.668195,2.474294,3.193901
L6_UPC,6.219487,3.628422,2.591064


In [8]:
chart.node_position

,source,target
L1_SBC,"(0.9, 1.156852772159361)","(1.0, 3.5053294588143933)"
L23_BC,"(0.9, 6.106423736514474)","(1.0, 8.463624372058986)"
L23_MC,"(0.9, 10.961853664066256)","(1.0, 13.560721522751422)"
L4_TPC,"(0.9, 16.969963515879787)","(1.0, 19.6380919336356)"
L5_TPC:A,"(0.9, 22.22969176172446)","(1.0, 25.06378911485109)"
L6_UPC,"(0.9, 28.736749959111933)","(1.0, 31.84649323169501)"


In [9]:
chart.node_data

position                                 size  \
                             source                     target     total   
L1_SBC     (0.9, 1.156852772159361)  (1.0, 3.5053294588143933)  4.696953   
L23_BC     (0.9, 6.106423736514474)   (1.0, 8.463624372058986)  4.714401   
L23_MC    (0.9, 10.961853664066256)  (1.0, 13.560721522751422)  5.197736   
L4_TPC    (0.9, 16.969963515879787)    (1.0, 19.6380919336356)  5.336257   
L5_TPC:A   (0.9, 22.22969176172446)   (1.0, 25.06378911485109)  5.668195   
L6_UPC    (0.9, 28.736749959111933)   (1.0, 31.84649323169501)  6.219487   

                                  flow                      
            source    target  outgoing  incoming     total  
L1_SBC    2.313706  2.383248  2.313706  2.383248  4.696953  
L23_BC    2.295342  2.419059  2.295342  2.419059  4.714401  
L23_MC    2.053800  3.143935  2.053800  3.143935  5.197736  
L4_TPC    3.150950  2.185307  3.150950  2.185307  5.336257  
L5_TPC:A  2.474294  3.193901  2.474294  3.193901  5.668195  
L6_UPC    3.628422  2.591064  3.628422  2.591064  6.219487

In [ ]:
node_one = chart.node_data.iloc[0]
node_one.position.source

In [ ]:
PolarPoint(
    **node_one.position.source)

In [ ]:
pd.concat([chart.node_flow], axis=1, keys=["flow"])

In [ ]:
def pr(**kw):
    print(kw)
pr(**chart.node_position.source.loc["L1_SBC"])

In [ ]:
pd.concat(
    [chart.node_size, chart.node_position, chart.node_flow],
    axis=1,
    keys=["size", "position", ("flow", "")]
)